In [1]:
from time import sleep, time
from subprocess import run
from pathlib import Path
from PIL import ImageColor
import pandas as pd
from IPython.display import display, Markdown, Latex, HTML
from remote_execution import RemoteExecution

# Connect to unreal

In [13]:
remote_exec = RemoteExecution()
remote_exec.start()
sleep(0.5)
for node in remote_exec.remote_nodes:
    remote_exec.open_command_connection(node.get("node_id"))


In [15]:

sleep(1)
remote_exec.has_command_connection()


True

# Load ldraw

In [16]:
ldraw_path = Path(R'C:\Program Files\Studio 2.0\ldraw')

In [17]:
colors = Path(R'C:\Program Files\Studio 2.0\data\StudioColorDefinition.txt')
df = pd.read_csv(colors, delimiter='	')

In [18]:
df = df[['LDraw Color Code', 'RGB value', 'Alpha', 'Ins_CMYK']]

In [20]:
copy_to_ue_tool = []
blander_exe = R'C:\Program Files\Blender Foundation\Blender 3.1\blender.exe'
file_to_import = R'c:\Other\Lego\test.ldr'
with open(file_to_import) as f:
    for line in f.readlines():
        if line[0] == '1':
            brick = line.split()  # color 1, file 14
            color = brick[1]
            vector = ','.join(brick[2:5])
            matrix = ','.join(brick[5:14])
            brick = brick[14].split('.')[0]
            color_data = df[df['LDraw Color Code'] == int(color)]
            try:
                color_values = ImageColor.getcolor(
                    color_data['RGB value'].iat[0], "RGB")
            except:
                print(f'skiped color {int(color)}')
            color_r = color_values[0]/255.0
            color_g = color_values[1]/255.0
            color_b = color_values[2]/255.0
            alpha = color_data['Alpha'].iat[0]
            place_lego_part = Path('place_lego_part.py')
            import_script = Path('import_script.py')
            import_fbx = Path('import_fbx.py')
            place_lego_part = Path('place_lego_part.py')
            import_materials = Path('import_materials.py')
            brick_path = Path().joinpath('models', f'{brick}.fbx')
            result = remote_exec.run_command(f'{place_lego_part.resolve()} -b "{brick}" -r " {matrix}" -v " {vector}" -m {color}')
            result_material = remote_exec.run_command(f'{import_materials.resolve()} -r {color_r} -g {color_g} -b {color_b} -a {alpha} -n {color}')
            if result_material['success'] == False:
                print(brick, "can't make materials")
                break
            if result['success'] == False:
                print(brick, "can't place. importing...")
                blender_import = run(
                    [blander_exe,
                     '--background', '--python',
                     import_script.resolve(), '--', brick]
                )
                if blender_import.returncode != 0:
                    print('blender fail')
                    break
                result = remote_exec.run_command(f'{import_fbx.resolve()} -f {brick_path.resolve()}')
                if result['success'] == False:
                    print(brick, "can't import")
                    break
                result = remote_exec.run_command(f'{place_lego_part.resolve()} -b "{brick}" -r " {matrix}" -v " {vector}" -m {color}')
                if result['success'] == False:
                    print(brick, "can't place after import")
                    break
            copy_to_ue_tool.append(f'{place_lego_part.resolve()} -b "{brick}" -r " {matrix}" -v " {vector}" -m {color}')
with open('last_commands.txt', 'w') as f:
    f.write('\n\n'.join(copy_to_ue_tool))
